In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from plotly import express as px
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from embetter.grab import ColumnGrabber
from embetter.text import SentenceEncoder
from umap import UMAP

In [2]:
train = pd.read_csv('../data/05_model_input/train.csv')
test = pd.read_csv('../data/05_model_input/test.csv')

In [3]:
def create_rating(dataframe, rating_column):
    dataframe['rating'] = [
        'positivo' if x >= 4 else 'negativo' for x in dataframe[rating_column]
    ]
    return dataframe

In [4]:
def clean_text(dataframe, text_column):
    dataframe[text_column] = dataframe[text_column].str.lower()
    dataframe[text_column] = dataframe[text_column].str.replace(r'[^\w\s]+', '')
    dataframe[text_column] = dataframe[text_column].str.replace(r'\d+', '')
    dataframe[text_column] = dataframe[text_column].astype(str)
    return dataframe

In [5]:
train = create_rating(train, 'overall_rating')

In [6]:
train = clean_text(train, 'review_text')
test = clean_text(test, 'review_text')

In [7]:
X = pd.DataFrame(train['review_text'])
y = pd.DataFrame(train['rating'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

/home/bobcasta/.b2wreviews/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bobcasta/.b2wreviews/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/bobcasta/.b2wreviews/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [12]:
# show the encoded classes
label_encoder.classes_

# show the what the encoded classes are mapped to (0, 1, 2)
list(label_encoder.inverse_transform([0, 1]))

['negativo', 'positivo']

In [15]:
pipeline = make_pipeline(
    ColumnGrabber('review_text'), 
    SentenceEncoder('all-mpnet-base-v2'), 
    LogisticRegression()
)

In [ ]:
pipeline.fit(X_train, y_train)

/home/bobcasta/.b2wreviews/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('columngrabber', ColumnGrabber(colname='review_text')),
                ('sentenceencoder',
                 SentenceEncoder(device=device(type='cuda'),
                                 name='all-mpnet-base-v2')),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)
pipeline.score(X_test, y_test)

0.8254129729206027